In [3]:
from Bio import Entrez
import re

Entrez.email = "emirhan.acikgoez@hotmail.com"  # Always tell NCBI who you are

# Search PubMed for articles with "GC content of" or "G + C content of" in the title or abstract
abstracts_with_gc = []
for i in range(0, 200, 20):
    handle = Entrez.esearch(db="pubmed", term='("GC content of" OR "G + C content of") AND bacterial[Title/Abstract]', retstart=i, retmax=20)
    record = Entrez.read(handle)
    ids = record["IdList"]
    # Process the IDs

    # Get the list of article IDs
    article_ids = record["IdList"]

    # Fetch the abstracts for the articles
    handle = Entrez.efetch(db="pubmed", id=article_ids, rettype="abstract")
    records = Entrez.read(handle)

    # Parse the abstracts
    for record in records["PubmedArticle"]:
        try:
            temp_record = record["MedlineCitation"]["Article"]
            abstract = temp_record["Abstract"]["AbstractText"][0]
            if re.search(r"\bGC\scontent\sof\b", abstract, re.IGNORECASE) and len(re.findall(r"([\d\.]+\%)", abstract)) == 1:
                bacterial_match = re.search(r"<i>(.*?)</i>", abstract)
                doi = record['PubmedData']['ArticleIdList'][0] if 'ArticleIdList' in record['PubmedData'] else 'N/A'
                if bacterial_match:
                    bacterial_name = bacterial_match.group(1)
                    # Prompt the user to input whether the GC content value is correct or not
                    print(f"DOI: {doi}")
                    correct = input(f"Is the GC content value 'GC content of' correct for {bacterial_name}? (y/n) ")
                    while correct.lower() not in ['y', 'n']:
                        correct = input("Invalid input. Please enter 'y' or 'n': ")
                    if correct.lower() == 'y':
                        abstracts_with_gc.append((bacterial_name, 'GC content of'))
        except KeyError:
            print("No abstract available")

# Print the bacterial names and their corresponding GC content phrases
for bacterial_name, gc_content in abstracts_with_gc:
    print(f"{bacterial_name} has the phrase 'GC content of'")


DOI: 37170421


KeyboardInterrupt: Interrupted by user